# Many to Many Relationships

Think about a Human Resources database for a large company

* Departments
* Employees
* Titles
* Managers
* Salaries


## Basic LDS

![](company_simple.png)

## Eliminating Many-Many relationship

![](company_intermediate.png)

## From Model to Table

```
employees=# \d departments
          Table "public.departments"
  Column   |         Type          | Modifiers
-----------+-----------------------+-----------
 dept_no   | character(4)          | not null
 dept_name | character varying(40) | not null


employees=# \d dept_emp
       Table "public.dept_emp"
  Column   |     Type     | Modifiers
-----------+--------------+-----------
 emp_no    | integer      | not null
 dept_no   | character(4) | not null
 from_date | date         | not null
 to_date   | date         | not null


employees=# \d employees
            Table "public.employees"
   Column   |         Type          | Modifiers
------------+-----------------------+-----------
 emp_no     | integer               | not null
 birth_date | date                  | not null
 first_name | character varying(14) | not null
 last_name  | character varying(16) | not null
 gender     | character varying(1)  | not null
 hire_date  | date                  | not null

```


## The Final LDS

![](company_advanced.png)

In [1]:
import warnings
warnings.filterwarnings('ignore')


In [2]:
%load_ext sql


In [3]:
%sql postgresql://yasiro01:@localhost/employees


'Connected: yasiro01@employees'

In [4]:
%%sql

select *
from departments natural join dept_emp natural join employees natural join salaries natural join titles
limit 10

10 rows affected.


emp_no,from_date,to_date,dept_no,dept_name,birth_date,first_name,last_name,gender,hire_date,salary,title
13021,1991-10-18,1992-03-19,d005,Development,1957-03-10,Yurii,Schaap,M,1991-10-18,45553,Senior Engineer
15083,1996-05-30,1997-04-19,d006,Quality Management,1958-11-24,Denis,Nicolson,M,1994-03-02,68084,Senior Engineer
17091,1992-02-11,1992-03-20,d004,Production,1958-11-08,Gunilla,Merkl,F,1988-01-30,48099,Assistant Engineer
19343,1986-06-02,1986-11-18,d007,Sales,1953-10-12,Heping,Hempstead,F,1986-06-02,44351,Staff
20910,1995-12-09,1996-01-07,d007,Sales,1963-08-06,Chenyi,Kavraki,M,1993-08-14,76194,Senior Staff
25281,1993-07-27,1993-11-08,d003,Human Resources,1956-11-10,Fumiyo,Delgrande,F,1991-06-07,47478,Staff
27762,1991-04-09,1991-08-14,d007,Sales,1954-06-06,Boalin,Griswold,F,1988-06-05,57852,Staff
28155,1995-10-09,1996-08-22,d002,Finance,1961-09-01,Jianwen,Marsiglia,F,1990-01-11,74637,Staff
31972,1993-01-21,1994-01-10,d005,Development,1952-12-18,Yifei,Kawashimo,F,1990-05-12,68011,Senior Engineer
32671,1998-11-05,1998-11-12,d003,Human Resources,1963-03-31,Atreye,Tetzlaff,M,1996-12-22,40000,Staff


## How many employees currently work in various departments?


In [9]:
%%sql

select * from dept_emp
where to_date > now()
limit 10


10 rows affected.


emp_no,dept_no,from_date,to_date
10001,d005,1986-06-26,9999-01-01
10002,d007,1996-08-03,9999-01-01
10003,d004,1995-12-03,9999-01-01
10004,d004,1986-12-01,9999-01-01
10005,d003,1989-09-12,9999-01-01
10006,d005,1990-08-05,9999-01-01
10007,d008,1989-02-10,9999-01-01
10009,d006,1985-02-18,9999-01-01
10010,d006,2000-06-26,9999-01-01
10012,d005,1992-12-18,9999-01-01


In [6]:
%%sql

select dept_name, count(*)
from departments natural join dept_emp
where to_date > now()
group by dept_name
order by count;

9 rows affected.


dept_name,count
Finance,12437
Human Resources,12898
Quality Management,14546
Marketing,14842
Research,15441
Customer Service,17569
Sales,37701
Production,53304
Development,61386


1. **Note:** the ``now()`` function is just a convenience for manually entereing the current date
2. **Note:** we can use aggregates without a grouby, realizing that said aggregate applies to the whole result

In [11]:
%%sql

select count(*)
from departments natural join dept_emp natural join employees natural join salaries natural join titles


1 rows affected.


count
7535


## Find all managers of the *Marketing* department

In [15]:
%%sql

select *
from departments natural join dept_manager natural join employees
where dept_name = 'Marketing';


2 rows affected.


emp_no,dept_no,dept_name,from_date,to_date,birth_date,first_name,last_name,gender,hire_date
110022,d001,Marketing,1985-01-01,1991-10-01,1956-09-12,Margareta,Markovitch,M,1985-01-01
110039,d001,Marketing,1991-10-01,9999-01-01,1963-06-21,Vishwani,Minakawa,M,1986-04-12


## Find *current* manager of every department

In [18]:
%%sql

select dept_name, first_name, last_name, from_date
from departments natural join dept_manager natural join employees
where to_date > now()
order by dept_name;


9 rows affected.


dept_name,first_name,last_name,from_date
Customer Service,Yuchang,Weedman,1996-01-03
Development,Leon,DasSarma,1992-04-25
Finance,Isamu,Legleitner,1989-12-17
Human Resources,Karsten,Sigstam,1992-03-21
Marketing,Vishwani,Minakawa,1991-10-01
Production,Oscar,Ghazalie,1996-08-30
Quality Management,Dung,Pesch,1994-06-28
Research,Hilary,Kambil,1991-04-08
Sales,Hauke,Zhang,1991-03-07


## Find all titles within the company

In [20]:
%%sql

select distinct title
from titles;


7 rows affected.


title
Technique Leader
Senior Engineer
Staff
Assistant Engineer
Engineer
Senior Staff
Manager


## Question!

Let's start with all the records

In [21]:
%%sql

select *
from departments natural join dept_emp natural join employees natural join salaries
limit 10;


10 rows affected.


emp_no,from_date,to_date,dept_no,dept_name,birth_date,first_name,last_name,gender,hire_date,salary
286829,1994-03-14,1995-01-11,d001,Marketing,1954-11-10,Kazuhisa,Vecchio,F,1986-03-17,48149
476388,1995-09-21,1995-11-28,d001,Marketing,1960-07-01,Elvis,Kroll,M,1991-10-13,65007
210440,1996-09-18,1996-11-10,d001,Marketing,1964-07-11,Chandrasekaran,Vernadat,M,1994-06-10,49760
465625,1996-10-04,1997-07-07,d001,Marketing,1961-10-02,Claudi,Piveteau,M,1996-10-04,49259
19196,2000-01-28,2000-05-12,d001,Marketing,1956-08-10,Leucio,Sury,M,1988-09-03,66890
259294,1991-02-07,1991-05-17,d001,Marketing,1962-03-13,Demos,Peyn,F,1988-01-26,66971
80329,1999-08-06,1999-09-05,d001,Marketing,1959-09-09,Moon,Ponthieu,F,1988-11-07,40000
268194,1994-05-27,1994-06-29,d001,Marketing,1960-02-12,Gila,Aamodt,F,1994-04-06,70796
294469,1995-08-17,1996-05-22,d001,Marketing,1960-07-18,Tetsushi,Biran,M,1987-06-21,68907
12505,1999-08-24,2000-08-23,d001,Marketing,1952-08-24,Barton,Goldhammer,M,1991-04-20,40423


## Let's look closer at *Barton Goldhammer*

In [23]:
%%sql

select *
from departments natural join dept_emp natural join employees
where employees.first_name = 'Barton' and employees.last_name = 'Goldhammer';


2 rows affected.


emp_no,dept_no,dept_name,from_date,to_date,birth_date,first_name,last_name,gender,hire_date
12505,d001,Marketing,1999-08-24,2000-08-23,1952-08-24,Barton,Goldhammer,M,1991-04-20
12505,d007,Sales,2000-08-23,2002-02-18,1952-08-24,Barton,Goldhammer,M,1991-04-20


## Let's find out how *Barton Goldhammer's* salary changed, if at all

In [31]:
%%sql

select *
from departments natural join dept_emp natural join employees natural join salaries
where employees.first_name = 'Barton' and employees.last_name = 'Goldhammer';


1 rows affected.


emp_no,from_date,to_date,dept_no,dept_name,birth_date,first_name,last_name,gender,hire_date,salary
12505,1999-08-24,2000-08-23,d001,Marketing,1952-08-24,Barton,Goldhammer,M,1991-04-20,40423


## Huh?

## A more specific version of JOIN operator

In [33]:
%%sql

select *
from departments natural join dept_emp natural join employees join salaries on employees.emp_no = salaries.emp_no
where employees.first_name = 'Barton' and employees.last_name = 'Goldhammer'
order by salaries.from_date;


6 rows affected.


emp_no,dept_no,dept_name,from_date,to_date,birth_date,first_name,last_name,gender,hire_date,emp_no_1,salary,from_date_1,to_date_1
12505,d001,Marketing,1999-08-24,2000-08-23,1952-08-24,Barton,Goldhammer,M,1991-04-20,12505,40423,1999-08-24,2000-08-23
12505,d007,Sales,2000-08-23,2002-02-18,1952-08-24,Barton,Goldhammer,M,1991-04-20,12505,40423,1999-08-24,2000-08-23
12505,d001,Marketing,1999-08-24,2000-08-23,1952-08-24,Barton,Goldhammer,M,1991-04-20,12505,43758,2000-08-23,2001-08-23
12505,d007,Sales,2000-08-23,2002-02-18,1952-08-24,Barton,Goldhammer,M,1991-04-20,12505,43758,2000-08-23,2001-08-23
12505,d001,Marketing,1999-08-24,2000-08-23,1952-08-24,Barton,Goldhammer,M,1991-04-20,12505,43942,2001-08-23,2002-02-18
12505,d007,Sales,2000-08-23,2002-02-18,1952-08-24,Barton,Goldhammer,M,1991-04-20,12505,43942,2001-08-23,2002-02-18


## What are the min, max, and average salaries per department?

[PostgreSQL: Documentation: 9.5: Data Type Formatting Functions](https://www.postgresql.org/docs/9.5/static/functions-formatting.html)


In [36]:
%%sql

select dept_name, min(salary), to_char(avg(salary), '999,999.99'), max(salary)
from departments natural join dept_emp natural join employees natural join salaries
group by dept_name;


9 rows affected.


dept_name,min,to_char,max
Customer Service,40000,"43,684.35",91638
Development,40000,"49,085.11",100285
Finance,40000,"61,479.84",105985
Human Resources,40000,"43,069.74",73900
Marketing,40000,"59,373.39",100242
Production,40000,"49,057.93",98370
Quality Management,40000,"46,026.20",79334
Research,40000,"49,130.98",81790
Sales,40000,"69,663.26",112513


```
select * from
dept_emp natural join salaries
```

```
emp_no  dept_no from to salary
1       2       X    Y  10
```


```
select * from
dept_emp join salaries on dept_emp.emp_no = salaries.emp_no
```



```
emp_no   dept_no  from_1  to_1 from_2 to_2 salary
1        2        X       Y    X      Y    10
1        3        Y       X    X      Y    10
2        1        X       Y    Y      Z    12

```



## Find the salary history for Holgard Pena


In [ ]:
%%sql


## Find the job title history for Holgard Pena


In [ ]:
%%sql


## Current job title counts per department


In [ ]:
%%sql


## Average salary by job title  - based on current salary and current title


In [ ]:
%%sql


## Who has held the most titles?


In [ ]:
%%sql
